# Show3DVolume -- Quick Demo

3D volume viewer with interactive orthogonal slices and WebGL ray-cast volume rendering.
Synthetic gold nanoparticle tomographic reconstruction with faceted shape, twin boundaries, and lattice fringes.

In [ ]:
%env ANYWIDGET_HMR=1

In [ ]:
import numpy as np
from quantem.widget import Show3DVolume


def make_gold_nanoparticle(n=96):
    """Realistic HAADF-STEM tomographic reconstruction of a multiply-twinned gold nanoparticle."""
    z, y, x = np.mgrid[:n, :n, :n]
    cz, cy, cx = n / 2, n / 2, n / 2
    ax, ay, az = np.abs(x - cx), np.abs(y - cy), np.abs(z - cz)

    # Truncated octahedron shape (faceted, common for FCC gold nanoparticles)
    # Blend of L1 norm (octahedron) and Linf norm (cube) → truncated octahedron
    l1 = ax + ay + az
    linf = np.maximum(np.maximum(ax, ay), az)
    shape_dist = np.maximum(l1 / 1.6, linf)
    R = n * 0.37
    particle = 1.0 / (1 + np.exp((shape_dist - R) * 1.2))  # smooth boundary

    # Z-contrast density: core slightly brighter (bimetallic Au@Pd-like)
    r = np.sqrt((x - cx) ** 2 + (y - cy) ** 2 + (z - cz) ** 2)
    core_weight = 1.0 / (1 + np.exp((r - R * 0.45) * 0.6))
    density = 0.65 + 0.35 * core_weight

    # Lattice fringes (111 planes, ~2.35 A spacing for Au)
    d111 = 3.8  # spacing in voxels
    fringes = 0.12 * (
        np.cos(2 * np.pi * (x + y + z) / (d111 * np.sqrt(3)))
        + 0.5 * np.cos(2 * np.pi * (x - y + z) / (d111 * np.sqrt(3)))
    )

    # Twin boundary (planar mirror defect at ~54.7 deg — common five-fold twin)
    twin1 = 0.18 * np.exp(-((x - cx) * 0.707 + (y - cy) * 0.707) ** 2 / 2.0)
    twin2 = 0.12 * np.exp(-((x - cx) * 0.707 - (z - cz) * 0.707) ** 2 / 2.0)

    # Stacking fault (planar defect)
    sf = 0.10 * np.exp(-((z - cz) + 0.3 * (x - cx)) ** 2 / 1.5)

    volume = particle * (density + fringes + twin1 + twin2 + sf)

    # Internal voids (vacancy clusters — common in real tomograms)
    for dx, dy, dz, vr in [(10, -6, 4, 3.0), (-8, 8, -5, 2.5), (5, -3, -12, 2.0)]:
        d = np.sqrt((x - cx - dx) ** 2 + (y - cy - dy) ** 2 + (z - cz - dz) ** 2)
        volume -= 0.5 * particle * np.exp(-(d ** 2) / (2 * vr ** 2))

    # Satellite particles on the surface (smaller Au clusters)
    for sx, sy, sz, sr in [
        (R * 0.85, 0, R * 0.5, 3.5),
        (-R * 0.7, R * 0.6, 0, 2.8),
        (0, -R * 0.8, -R * 0.5, 3.0),
    ]:
        d = np.sqrt((x - cx - sx) ** 2 + (y - cy - sy) ** 2 + (z - cz - sz) ** 2)
        volume += 0.8 / (1 + np.exp((d - sr) * 2.0))

    # Realistic reconstruction noise (Poisson-like)
    volume += np.random.normal(0, 0.015, volume.shape)
    volume = np.clip(volume, 0, None)
    return volume.astype(np.float32)


volume = make_gold_nanoparticle()
Show3DVolume(volume, title="Gold Nanoparticle — Tomographic Reconstruction", cmap="inferno")